# Generate a Pandas data frame with entities with a given attribute

Issue[#13](https://github.com/time-link/timelink-py/issues/13)

In [1]:
# NBVAL_IGNORE_OUTPUT
from timelink.notebooks import TimelinkNotebook

tlnb = TimelinkNotebook(db_name="issue13")
tlnb.print_info()

Timelink version: 1.1.27
Project name: test-project
Project home: /Users/jrc/develop/timelink-py-worktree/main/tests/timelink-home/projects/test-project
Database type: sqlite
Database name: issue13
Kleio image: timelinkserver/kleio-server
Kleio server token: 4zb74...
Kleio server URL: http://127.0.0.1:8088
Kleio server home: /Users/jrc/develop/timelink-py-worktree/main/tests/timelink-home/projects/test-project
Kleio server container: hopeful_kalam
Kleio version requested: latest
Kleio server version: 12.9.588 (2025-06-06 16:04:22)
SQLite directory: /Users/jrc/develop/timelink-py-worktree/main/tests/timelink-home/projects/test-project/database/sqlite
Database version: 6ccf1ef385a6
Call print_info(show_token=True) to show the Kleio Server token
Call print_info(show_password=True) to show the Postgres password
TimelinkNotebook(project_name=test-project, project_home=/Users/jrc/develop/timelink-py-worktree/main/tests/timelink-home/projects/test-project, db_type=sqlite, db_name=issue13, kle

### Import file

In [2]:
# NBVAL_IGNORE_OUTPUT

db = tlnb.db
test_files_path = "sources/reference_sources/linked_data/"
db.update_from_sources(test_files_path)
tlnb.get_import_status(path=test_files_path)[['name', 'path','import_status','import_errors','import_warnings']]

/Users/jrc/develop/timelink-py-worktree/main/timelink/api/models/pom_som_mapper.py:697: UserWarning: Ambiguous mapping! more than one element of rel is mapped to column class date check mapping for this group
  element: KElement = group.get_element_by_name_or_class(cattr.colclass)


,name,path,import_status,import_errors,import_warnings
0,dehergne-a.cli,sources/reference_sources/linked_data/dehergne...,E,3,0
1,dehergne-locations-1644.cli,sources/reference_sources/linked_data/dehergne...,I,0,0


### Show imported source

In [3]:
# NBVAL_IGNORE_OUTPUT
from timelink.api.models import Source
import warnings
warnings.filterwarnings('always')

with tlnb.db.session() as session:
    sources = session.query(Source).order_by(Source.id).all()
    for source in sources:
        print("\n".join(source.to_kleio().split()[:10]))

/Users/jrc/develop/timelink-py-worktree/main/timelink/api/models/attribute.py:58: UserWarning: Invalid date: 16553023 -> unconverted data remains: 3
  r += f"{self.for_kleio('the_date')}"
/Users/jrc/develop/timelink-py-worktree/main/timelink/api/models/attribute.py:58: UserWarning: Invalid date: 16553023 -> unconverted data remains: 3
  r += f"{self.for_kleio('the_date')}"
/Users/jrc/develop/timelink-py-worktree/main/timelink/api/models/attribute.py:58: UserWarning: Invalid date: 16553023 -> unconverted data remains: 3
  r += f"{self.for_kleio('the_date')}"
/Users/jrc/develop/timelink-py-worktree/main/timelink/api/models/relation.py:74: UserWarning: Missing destination for relation
 Relation(id=Entity(id="deh-luis-de-almeida-rela44", pom_class="relation", inside="deh-luis-de-almeida", the_source=dehergne-a, the_order=655, the_level=5, the_line=541, groupname="relation", updated=2026-01-22 06:35:59.055713, indexed=None), origin="deh-luis-de-almeida-ref2", destination="deh-belchior-migue

fonte$dehergne-a/0/type=Dicionário
Biográfico/ref=None/loc=Online
archive.org:details:bhsi37/kleiofile="/kleio-home/sources/reference_sources/linked_data/dehergne-a.cli""/replaces=None/obs="""
Dehergne,
Joseph,
Répertoire
des
Jésuites
de
Chine,
fonte$dehergne-locations-1644/0/type=geoinformation/ref=None/loc=None/kleiofile="/kleio-home/sources/reference_sources/linked_data/dehergne-locations-1644.cli""/replaces=None/obs="""
Planche:
Carte
des
Chrétientés
Chinoises
de
la
fin
des


## Use entities from attributes

In [4]:
from timelink.pandas import entities_with_attribute

# Get list of people with with a certain value in a specific attribute
df = entities_with_attribute(
                    entity_type='person',
                    the_type='estadia',
                    the_value='Macau',
                    dates_in=(1580, 1644),
                    show_elements=['name'],
                    more_attributes=['nacionalidade','jesuita-entrada'],
                    db=tlnb.db,
                    sql_echo=True)
df.info()

Query for estadia:
 SELECT persons.id, persons.name, eattributes.attr_id AS "estadia.attr_id", eattributes.the_type AS "estadia.type", eattributes.the_value AS estadia, eattributes.the_date AS "estadia.date", eattributes.a_the_line AS "estadia.line", eattributes.a_the_level AS "estadia.level", eattributes.a_groupname AS "estadia.groupname", eattributes.aobs AS "estadia.obs", eattributes.a_extra_info AS "estadia.extra_info" 
FROM entities JOIN persons ON entities.id = persons.id JOIN eattributes ON eattributes.entity = persons.id 
WHERE eattributes.the_type LIKE :the_type_1 AND eattributes.the_value LIKE :the_value_1 AND eattributes.the_date > :the_date_1 AND eattributes.the_date < :the_date_2 ORDER BY eattributes.the_date
Query for more_attributes=nacionalidade:
 SELECT eattributes.entity AS id, eattributes.the_value AS nacionalidade, eattributes.the_date AS "nacionalidade.date", eattributes.aobs AS "nacionalidade.obs", eattributes.a_extra_info AS "nacionalidade.extra_info" 
FROM eattr

In [5]:
df[["name","nacionalidade","jesuita-entrada", "estadia","estadia.date","jesuita-entrada"]]

,name,nacionalidade,jesuita-entrada,estadia,estadia.date,jesuita-entrada
id,,,,,,
deh-luis-de-almeida-ref2,Belchior Miguel Carneiro Leitão,NaN,NaN,Macau,15800000,NaN
deh-antonio-de-almeida,António de Almeida,Portugal,?,Macau,15850731,?
deh-diogo-antunes,Diogo Antunes,Portugal,Évora,Macau,15870000,Évora
deh-antonio-de-almeida,António de Almeida,Portugal,?,Macau,15900000,?
deh-antonio-de-almeida,António de Almeida,Portugal,?,Macau,15910900,?
deh-giulio-aleni,Giulio Aleni,Itália,Novellara,Macau,16100000,Novellara
deh-giulio-aleni,Giulio Aleni,Itália,Novellara,Macau,16390000,Novellara
deh-gaspar-do-amaral,Gaspar do Amaral,Portugal,?,Macau,16400000,?
deh-gaspar-do-amaral,Gaspar do Amaral,Portugal,?,Macau,16420000,?


In [6]:
# NBVAL_IGNORE_OUTPUT
from timelink.pandas import entities_with_attribute

# Get list of people with with a certain value in a specific attribute
df = entities_with_attribute(
                    entity_type='person',
                    the_type='nacionalidade',
                    the_value='Portugal',
                    show_elements=['name','sex'],
                    more_attributes=['nascimento','jesuita-entrada','wicky'],
                    name_like='%Almeida%',
                    db=tlnb.db,
                    sql_echo=True)
# df.info()

Query for nacionalidade:
 SELECT persons.id, persons.name, persons.sex, eattributes.attr_id AS "nacionalidade.attr_id", eattributes.the_type AS "nacionalidade.type", eattributes.the_value AS nacionalidade, eattributes.the_date AS "nacionalidade.date", eattributes.a_the_line AS "nacionalidade.line", eattributes.a_the_level AS "nacionalidade.level", eattributes.a_groupname AS "nacionalidade.groupname", eattributes.aobs AS "nacionalidade.obs", eattributes.a_extra_info AS "nacionalidade.extra_info" 
FROM entities JOIN persons ON entities.id = persons.id JOIN eattributes ON eattributes.entity = persons.id 
WHERE eattributes.the_type LIKE :the_type_1 AND eattributes.the_value LIKE :the_value_1 AND persons.name LIKE :name_1 ORDER BY eattributes.the_date
Query for more_attributes=nascimento:
 SELECT eattributes.entity AS id, eattributes.the_value AS nascimento, eattributes.the_date AS "nascimento.date", eattributes.aobs AS "nascimento.obs", eattributes.a_extra_info AS "nascimento.extra_info" 


In [7]:
df[["name", "nacionalidade","nascimento","jesuita-entrada"]]

,name,nacionalidade,nascimento,jesuita-entrada
id,,,,
deh-afonso-aires-ref1,Aires de Almeida,Portugal,Santarém,?
deh-antonio-de-almeida,António de Almeida,Portugal,"Trancoso, diocese de Viseu",?
deh-jose-de-almeida-i,José de Almeida,Portugal,NaN,NaN
deh-jose-de-almeida-ii,José de Almeida,Portugal,Lisboa,?
deh-jose-de-almeida-iii,José de Almeida,Portugal,"Arcos, diocese de Coimbra",?
deh-jose-bernardo-de-almeida,José Bernardo de Almeida,Portugal,"Penela, diocese de Coimbra",Lisboa
deh-jose-bernardo-de-almeida,José Bernardo de Almeida,Portugal,"Penela, diocese de Coimbra",Lisboa
deh-jose-estevao-de-almeida,José Estêvão de Almeida,Portugal,"Esgueira, diocese de Coimbra",?
deh-luis-de-almeida,Luís de Almeida,Portugal,Lisboa,Japão
